In [ ]:
from scapy.all import rdpcap, UDP, IP
import datetime
import re

pcap_file = "measurements/test1.pcap"

packets = rdpcap(pcap_file)

entry_timestamps = []
packet_timestamps = []

for pkt in packets:
    if IP in pkt and UDP in pkt:
        if pkt[IP].dst == "10.0.0.1":
            ts = pkt.time
            ts = float(pkt.time)
            sec = int(ts)
            nsec = int((ts - sec) * 1_000_000_000)

            formatted = f"{datetime.datetime.fromtimestamp(sec).strftime('%H:%M:%S')}.{nsec:09d}"
            packet_timestamps.append(formatted)

with open("measurements/entries_1.txt") as f:
    lines = f.readlines()
    add_lines = []
    timestamps = []
    for line in lines:
        if "Adding entry at:" in line:
            add_lines.append(line.strip())
    for line in add_lines:
        match = re.search(r"Adding entry at:\s+(\d{2}:\d{2}:\d{2}\.\d+)", line)
        if match:
            timestamp = match.group(1)
            #print("Extracted timestamp:", timestamp)
            timestamps.append(timestamp)
        else:
            print("No timestamp found")
    #print(timestamps)
    entry_timestamps = timestamps
print("Packet timestamps:", packet_timestamps)

Extracted timestamp: 11:49:52.335251194
Extracted timestamp: 11:49:54.335605969
Extracted timestamp: 11:49:56.335997483
Extracted timestamp: 11:49:58.336421776
Extracted timestamp: 11:50:00.336810628
Extracted timestamp: 11:50:02.337205529
Extracted timestamp: 11:50:04.337636006
Extracted timestamp: 11:50:06.338025878
Extracted timestamp: 11:50:08.338452109
Extracted timestamp: 11:50:10.338849622
Extracted timestamp: 11:50:12.339250370
Extracted timestamp: 11:50:14.339674385
Extracted timestamp: 11:50:16.340100844
Extracted timestamp: 11:50:18.340495656
Extracted timestamp: 11:50:20.340883351
Extracted timestamp: 11:50:22.341302046
Extracted timestamp: 11:50:24.341729213
Extracted timestamp: 11:50:26.342120530
Extracted timestamp: 11:50:28.342544024
Extracted timestamp: 11:50:30.342936291
Extracted timestamp: 11:50:32.343331478
Extracted timestamp: 11:50:34.343728434
Extracted timestamp: 11:50:36.344117907
Extracted timestamp: 11:50:38.344511284
Extracted timestamp: 11:50:40.344907955


In [20]:
def to_ns_timestamp(tstr):
    hh, mm, rest = tstr.split(":")
    ss, ns = rest.split(".")
    hh = int(hh)
    mm = int(mm)
    ss = int(ss)
    ns = int(ns)
    total_ns = ((hh*3600 + mm*60 + ss) * 1_000_000_000) + ns
    return total_ns


entry_ns = [to_ns_timestamp(t) for t in entry_timestamps]
packet_ns = [to_ns_timestamp(t) for t in packet_timestamps]

packet_ns.sort()

latencies = []

for entry_time in entry_ns:
    first_packet_after_entry = None

    for packet_time in packet_ns:
        if packet_time > entry_time:
            first_packet_after_entry = packet_time
            break

    if first_packet_after_entry is None:
        latencies.append(None)
        continue

    latency = first_packet_after_entry - entry_time
    latencies.append(latency)

print("\n--- Entry → First Packet Latencies ---")
for i, lat in enumerate(latencies):
    if lat is None:
        print(f"Entry {i}: kein Paket danach")
    else:
        print(f"Entry {i}: {lat/1_000_000:.3f} ms")

latencies_cleaned = [l for l in latencies if l is not None]
print(f"\nDurchschnittliche Latenz: {sum(latencies_cleaned)/len(latencies_cleaned)/1_000_000:.3f} ms")
print(f"Maximale Latenz: {max(latencies_cleaned)/1_000_000:.3f} ms")
print(f"Minimale Latenz: {min(latencies_cleaned)/1_000_000:.3f} ms")


--- Entry → First Packet Latencies ---
Entry 0: 32.072 ms
Entry 1: 2.421 ms
Entry 2: 2.376 ms
Entry 3: 0.976 ms
Entry 4: 0.928 ms
Entry 5: 0.895 ms
Entry 6: 1.148 ms
Entry 7: 1.442 ms
Entry 8: 1.325 ms
Entry 9: 1.640 ms
Entry 10: 0.251 ms
Entry 11: 0.520 ms
Entry 12: 2.090 ms
Entry 13: 1.046 ms
Entry 14: 0.656 ms
Entry 15: 3.605 ms
Entry 16: 3.891 ms
Entry 17: 2.146 ms
Entry 18: 2.419 ms
Entry 19: 0.688 ms
Entry 20: 1.040 ms
Entry 21: 1.289 ms
Entry 22: 2.209 ms
Entry 23: 2.169 ms
Entry 24: 0.783 ms
Entry 25: 0.737 ms
Entry 26: 0.683 ms
Entry 27: 0.938 ms
Entry 28: 1.231 ms
Entry 29: 1.161 ms
Entry 30: 1.476 ms
Entry 31: 3.419 ms
Entry 32: 0.407 ms
Entry 33: 1.987 ms
Entry 34: 0.948 ms
Entry 35: 0.479 ms
Entry 36: 3.467 ms
Entry 37: 3.779 ms
Entry 38: 2.038 ms
Entry 39: 2.329 ms
Entry 40: 0.581 ms
Entry 41: 0.906 ms
Entry 42: 1.192 ms
Entry 43: 2.083 ms
Entry 44: 2.058 ms
Entry 45: 0.676 ms
Entry 46: 0.646 ms
Entry 47: 0.602 ms
Entry 48: 0.892 ms
Entry 49: 1.187 ms
Entry 50: 1.075 ms
